In [20]:
import numpy as np 
import pandas as pd
import random

# For reproducibility
seed_value = 573
random.seed(seed_value)
np.random.seed(seed_value)

num_hours = 1008  # 42 days * 24 hours
patient_df = pd.DataFrame({'Absolute Hour': np.arange(1, num_hours + 1)})

# Derive time components
patient_df['Week'] = ((patient_df['Absolute Hour'] - 1) // 168) + 1
patient_df['Hour (Day)'] = ((patient_df['Absolute Hour'] - 1) % 24) + 1
patient_df['Hour (Week)'] = ((patient_df['Absolute Hour'] - 1) % 168) + 1

# --- Define parameters for the Floating pod's sinusoidal variation ---
offset_shift = 0.6129
amplitude = 15.5
phase = 10

floating_mean = 55 + amplitude * (
    np.sin(2 * np.pi / 24 * (patient_df['Hour (Day)'] - phase)) + offset_shift
)

patient_df['Floating'] = np.clip(
    np.random.normal(loc=floating_mean, scale=3).round().astype(int),
    0, None
)

# Generate counts for the other areas as before
patient_df['Purple'] = np.clip(
    np.random.normal(loc=15, scale=3, size=len(patient_df)).round().astype(int),
    0, None
)
patient_df['EDOU'] = np.clip(
    np.random.normal(loc=5, scale=2, size=len(patient_df)).round().astype(int),
    0, None
)

# --- Split the Floating count into subcounts for each position ---
positions_split = ["Red Pod", "Blue Pod", "Green Pod", "Orange Pod", "iTrack", "Hallway"]

def split_floating_count(total):
    # Sample the Red Pod count using a binomial draw with probability 1/21,
    # then cap it at 4.
    red_count = min(np.random.binomial(total, 1/21), 4)
    # Determine the remaining patients after assigning Red Pod.
    remaining = total - red_count
    # For the other five positions, assign counts using a multinomial draw
    # with equal probabilities.
    other_counts = np.random.multinomial(remaining, [0.2] * 5)
    # Return the array in the desired order.
    return np.array([red_count] + list(other_counts))

# Apply the splitting function row by row on the Floating column
split_counts = patient_df['Floating'].apply(split_floating_count)
split_df = pd.DataFrame(split_counts.tolist(), columns=positions_split)

# Join the new columns back to the main DataFrame
patient_df = patient_df.join(split_df)

# Save the DataFrame to CSV
patient_df.to_csv("patient_forecast.csv", index=False)
patient_df


,Absolute Hour,Week,Hour (Day),Hour (Week),Floating,Purple,EDOU,Red Pod,Blue Pod,Green Pod,Orange Pod,iTrack,Hallway
0,1,1,1,1,57,15,1,1,9,15,13,7,12
1,2,1,2,2,51,18,5,0,9,11,12,8,11
2,3,1,3,3,48,15,8,1,7,6,12,14,8
3,4,1,4,4,49,15,3,3,9,4,8,13,12
4,5,1,5,5,46,15,5,0,15,6,8,7,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,1004,6,20,164,78,15,3,1,16,20,17,12,12
1004,1005,6,21,165,64,12,4,2,11,15,11,11,14
1005,1006,6,22,166,65,24,3,3,21,12,7,8,14
1006,1007,6,23,167,62,15,7,2,11,15,12,12,10


In [21]:
# Define the patient count columns
patient_columns = ['Floating', 'Purple', 'EDOU', 'Red Pod', 'Blue Pod', 
                   'Green Pod', 'Orange Pod', 'iTrack', 'Hallway']

# Sum the values in each column
total_patients = patient_df[patient_columns].sum()

# Create a new DataFrame from the summed values
df_patient_count = pd.DataFrame({
    'Area': total_patients.index,
    'Total Patients': total_patients.values
})

# Optionally, save this summary DataFrame to a CSV file
df_patient_count.to_csv("patient_count.csv", index=False)

df_patient_count


,Area,Total Patients
0,Floating,65106
1,Purple,15253
2,EDOU,5150
3,Red Pod,2773
4,Blue Pod,12474
5,Green Pod,12462
6,Orange Pod,12431
7,iTrack,12474
8,Hallway,12492


In [22]:
import numpy as np
import pandas as pd

# Assuming patient_df is already created as in your original code

# Create a copy or just work on the existing DataFrame:
nurse_df = patient_df.copy()

# Calculate the number of nurses needed at each position
# Ratios:
#   Red Pod:    1 nurse per 2 patients
#   Purple:     1 nurse per 7 patients
#   iTrack:     1 nurse per 12 patients
#   Everywhere else (Blue Pod, Green Pod, Orange Pod, Hallway, EDOU): 1 nurse per 5 patients

nurse_df['Nurses Red Pod']    = np.ceil(nurse_df['Red Pod']    / 2).astype(int)
nurse_df['Nurses Purple']     = np.ceil(nurse_df['Purple']     / 7).astype(int)
nurse_df['Nurses iTrack']     = np.ceil(nurse_df['iTrack']     / 12).astype(int)
nurse_df['Nurses Blue Pod']   = np.ceil(nurse_df['Blue Pod']   / 5).astype(int)
nurse_df['Nurses Green Pod']  = np.ceil(nurse_df['Green Pod']  / 5).astype(int)
nurse_df['Nurses Orange Pod'] = np.ceil(nurse_df['Orange Pod'] / 5).astype(int)
nurse_df['Nurses Hallway']    = np.ceil(nurse_df['Hallway']    / 5).astype(int)
nurse_df['Nurses EDOU']       = np.ceil(nurse_df['EDOU']       / 5).astype(int)

# Now nurse_df has columns for nurse demand at each position.
# You can choose which columns you want in your final output.
# For example, keep time indicators plus nurse columns:

nurse_demand_cols = [
    'Absolute Hour', 
    'Week', 
    'Hour (Day)', 
    'Hour (Week)',
    'Nurses Red Pod',
    'Nurses Purple',
    'Nurses iTrack',
    'Nurses Blue Pod',
    'Nurses Green Pod',
    'Nurses Orange Pod',
    'Nurses Hallway',
    'Nurses EDOU'
]

final_nurse_df = nurse_df[nurse_demand_cols]

# Define which columns you want to sum for 'Nurses Floating'
floating_cols = [
    'Nurses Red Pod',
    'Nurses Blue Pod',
    'Nurses Green Pod',
    'Nurses Orange Pod',
    'Nurses iTrack',
    'Nurses Hallway'
]

# Then create 'Nurses Floating' by summing those columns across each row
final_nurse_df['Nurses Floating'] = final_nurse_df[floating_cols].sum(axis=1)



# Save to CSV if desired
final_nurse_df.to_csv('nurse_demand.csv', index=False)

# Optional: display or return `final_nurse_df` directly
final_nurse_df


C:\Users\Mustafa\AppData\Local\Temp\ipykernel_30388\1364521037.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_nurse_df['Nurses Floating'] = final_nurse_df[floating_cols].sum(axis=1)


,Absolute Hour,Week,Hour (Day),Hour (Week),Nurses Red Pod,Nurses Purple,Nurses iTrack,Nurses Blue Pod,Nurses Green Pod,Nurses Orange Pod,Nurses Hallway,Nurses EDOU,Nurses Floating
0,1,1,1,1,1,3,1,2,3,3,3,1,13
1,2,1,2,2,0,3,1,2,3,3,3,1,12
2,3,1,3,3,1,3,2,2,2,3,2,2,12
3,4,1,4,4,2,3,2,2,1,2,3,1,12
4,5,1,5,5,0,3,1,3,2,2,2,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,1004,6,20,164,1,3,1,4,4,4,3,1,17
1004,1005,6,21,165,1,2,1,3,3,3,3,1,14
1005,1006,6,22,166,2,4,1,5,3,2,3,1,16
1006,1007,6,23,167,1,3,1,3,3,3,2,2,13
